In [1]:
%load_ext autoreload
# %reload_ext autoreload

# Reload all modules imported with %aimport every time before executing the Python code typed
%autoreload 1

%aimport context_nn 
%aimport phrase_feeder 
%aimport notes
%aimport watch_point
%aimport cluster
import numpy as np
from bitarray import bitarray
from context_nn import ContextNN
from watch_point import WatchPoint
from cluster import Cluster
from phrase_feeder import PhraseFeeder
from pprint import pprint
import math
import constants as const

In [8]:
abc_notes = notes.Notes(note_count=26, 
                        notation_count=10, 
                        active_bits=8, 
                        bit_count=255)

In [2]:
import pickle

def load_phrase_base(file_name: str) -> (dict, list):
    with open(file_name, 'rb') as f:
        data = pickle.load(f)
        phrase_base = data.get('phrase_base', {})
        marks = data.get('marks', {})
        return phrase_base, marks

In [3]:
phrase_base, marks = load_phrase_base('./data/texts/phrase_base.pickle')

In [6]:
from ipythonblocks import BlockGrid

def draw_notation(notation: np.array):
    bit_grid = BlockGrid(len(notation), 1, fill=(17, 41, 129))
    for block in range(bit_grid.width):
        color = bit_grid[0, block]
        if notation[block]:
            bit_grid[0, block] = (244, 195, 173)
    bit_grid.lines_on = False
    bit_grid.show()

def draw_note_notations(notes, note_idx: int):
    for i in range(len(notes[note_idx])):
        draw_notation(notes.note_notation_as_bits(note_idx, i))

In [7]:
key = list(phrase_base.keys())[0]
phrase = phrase_base[key][0]
bit_chord = abc_notes.phrase_chord(phrase)
draw_notation(abc_notes.notation_as_bits(bit_chord))
print(phrase)
for note_idx, notation_idx in phrase:
    notation = abc_notes.note_notation_as_bits(note_idx, notation_idx)
    draw_notation(notation)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


[(8, 0), (0, 1), (13, 2), (24, 3), (18, 4)]


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [96]:
feeder = PhraseFeeder(phrase_base, marks)    

In [ ]:
phrases, output_bits = feeder.take_phrase_bunch()
print(output_bits, marks[output_bits])
print(phrases)

In [40]:
abc_notes = notes.Notes(note_count=26, 
                        notation_count=10, 
                        active_bits=8, 
                        bit_count=255)

feeder = phrase_feeder.PhraseFeeder(phrase_base, marks)  

cxnn = context_nn.ContextNN(input_bit_count=255,
                            output_bit_count=8,
                            watch_point_count=100,
                            watch_bit_count=32,
                            cluster_make_threshold=6,
                            cluster_activate_threshold=4)

In [99]:
len(cxnn.watch_points.keys())

100

In [8]:
%%time
phrases, output_bits = feeder.take_phrase_batch(count=200)
output_bits = set(output_bits)
for phrase in phrases:
    bit_chord = abc_notes.phrase_chord(phrase)
    cxnn.receive_bits(input_bits=bit_chord, output_bits=output_bits)
    

Wall time: 3.11 s


In [5]:
def feed_phrase_batch(feeder, count=200):
    phrases, output_bits = feeder.take_phrase_batch(count=200)
    output_bits = set(output_bits)
    for phrase in phrases:
        bit_chord = abc_notes.phrase_chord(phrase)
        cxnn.receive_bits(input_bits=bit_chord, output_bits=output_bits)

def feed_n_batches(feeder, n=10, batch_size=200):
    for i in range(n):
        feed_phrase_batch(feeder, count=batch_size)
        print(f'batch {i+1}/{n}')
    print(f'clusters: {cxnn.cluster_count()}')
        
def show_point_stats(cxnn: ContextNN):
    cluster_counts = [wp.cluster_count() for wp in cxnn.watch_points.values()]
    output_bits = [(wp.output_bit, len(wp.clusters.values())) 
                   for wp in cxnn.watch_points.values()]
    print('Cluster count:', sum(cluster_counts))
    print(output_bits)
  

In [ ]:
cxnn.state = const.STATE_LEARN

In [100]:
cxnn.state = const.STATE_CONSOLIDATE

In [44]:
%%time
feed_n_batches(feeder, n=10, batch_size=200)

batch 1/10
batch 2/10
batch 3/10
batch 4/10
batch 5/10
batch 6/10
batch 7/10
batch 8/10
batch 9/10
batch 10/10
clusters: 91045
Wall time: 17.6 s


In [52]:
show_point_stats(cxnn)
marks

Cluster count: 91045
[(0, 0), (1, 0), (6, 0), (2, 0), (1, 0), (2, 0), (7, 0), (2, 0), (3, 0), (2, 0), (2, 0), (3, 0), (0, 0), (2, 0), (5, 0), (7, 0), (4, 0), (4, 0), (1, 0), (3, 0), (5, 0), (1, 0), (5, 0), (1, 0), (5, 0), (2, 0), (4, 0), (5, 0), (2, 0), (5, 0), (6, 0), (7, 0), (2, 0), (0, 0), (5, 0), (0, 0), (4, 0), (6, 0), (2, 0), (3, 0), (7, 0), (0, 0), (1, 0), (2, 0), (2, 0), (3, 0), (0, 0), (4, 0), (0, 0), (3, 0), (5, 0), (5, 0), (4, 0), (7, 0), (1, 0), (6, 0), (3, 0), (1, 0), (4, 0), (5, 0), (5, 0), (1, 0), (4, 0), (4, 0), (6, 0), (6, 0), (6, 0), (5, 0), (5, 0), (0, 0), (0, 0), (3, 0), (2, 0), (4, 0), (7, 0), (0, 0), (2, 0), (6, 0), (5, 0), (7, 0), (3, 0), (5, 0), (4, 0), (0, 0), (7, 0), (6, 0), (5, 0), (4, 0), (1, 0), (1, 0), (6, 0), (1, 0), (7, 0), (3, 0), (7, 0), (2, 0), (7, 0), (3, 0), (5, 0), (5, 0)]


{(0, 1, 3, 6): 'jbo',
 (0, 2, 3, 5): 'rus',
 (0, 2, 3, 7): 'bel',
 (0, 2, 4, 6): 'blg',
 (1, 3, 4, 5): 'pol',
 (1, 4, 5, 7): 'eng',
 (2, 3, 5, 7): 'epo',
 (2, 4, 5, 6): 'ukr'}

In [46]:
def dict_value(d: dict, n=0) -> any:
    i = 0
    diter = iter(d)
    while i <= n:
        key = next(diter)
        i += 1
    return d[key]

In [49]:
point = dict_value(cxnn.watch_points)
cluster = point.cluster_objects[0]
# pprint(cluster.stats)
# pprint(cluster.component_stats())
pprint(cluster.bit_rate())

sorted(cluster.component_stats().items(), key=lambda x: x[1], reverse=True)

# parts = cluster.component_stats()
# enthropy = 0.0
# for part in parts.values():
#     enthropy += -part * math.log2(part)
# print(enthropy)

(array([ 1.        ,  0.625     ,  0.70833333,  0.58333333,  0.625     ,  1.        ]),
 [7, 35, 147, 157, 195, 230])


[((7, 147, 195, 230), 0.2222222222222222),
 ((7, 147, 157, 230), 0.18518518518518517),
 ((7, 35, 195, 230), 0.14814814814814814),
 ((7, 35, 157, 230), 0.1111111111111111),
 ((35, 157, 195, 230), 0.1111111111111111),
 ((7, 35, 147, 157), 0.07407407407407407),
 ((7, 35, 147, 230), 0.07407407407407407),
 ((7, 147, 157, 195, 230), 0.037037037037037035),
 ((7, 35, 147, 195), 0.037037037037037035)]

In [51]:
noize_clusters = 0
total_clusters = 0
cluster_lens = {}
cluster_acts = {}
for wp in cxnn.watch_points.values():
    for cluster in wp.cluster_objects:
        total_clusters += 1
        cluster_lens[len(cluster.bits)] = cluster_lens.get(len(cluster.bits), 0) + 1
        cluster_acts[len(cluster.stats)] = cluster_acts.get(len(cluster.stats), 0) + 1
        if not cluster.has_big_component(threshold=0.2, min_activations=10):
            noize_clusters += 1
print(total_clusters)
print(noize_clusters)
pprint(cluster_lens)
pprint(cluster_acts)


91045
63380
{6: 38268,
 7: 27473,
 8: 15237,
 9: 6665,
 10: 2489,
 11: 695,
 12: 178,
 13: 38,
 14: 2}
{0: 102,
 1: 1138,
 2: 2436,
 3: 3007,
 4: 3395,
 5: 3693,
 6: 3996,
 7: 4355,
 8: 4316,
 9: 4348,
 10: 4302,
 11: 4047,
 12: 3787,
 13: 3452,
 14: 3070,
 15: 2623,
 16: 2399,
 17: 2068,
 18: 1815,
 19: 1651,
 20: 1494,
 21: 1466,
 22: 1370,
 23: 1291,
 24: 1293,
 25: 1206,
 26: 1149,
 27: 1169,
 28: 1012,
 29: 989,
 30: 949,
 31: 842,
 32: 813,
 33: 735,
 34: 690,
 35: 689,
 36: 609,
 37: 620,
 38: 518,
 39: 516,
 40: 455,
 41: 451,
 42: 405,
 43: 398,
 44: 378,
 45: 393,
 46: 332,
 47: 356,
 48: 318,
 49: 314,
 50: 304,
 51: 294,
 52: 254,
 53: 256,
 54: 231,
 55: 258,
 56: 235,
 57: 207,
 58: 194,
 59: 181,
 60: 198,
 61: 165,
 62: 172,
 63: 187,
 64: 161,
 65: 165,
 66: 167,
 67: 142,
 68: 140,
 69: 135,
 70: 134,
 71: 111,
 72: 115,
 73: 112,
 74: 107,
 75: 113,
 76: 100,
 77: 106,
 78: 110,
 79: 91,
 80: 83,
 81: 68,
 82: 78,
 83: 81,
 84: 85,
 85: 79,
 86: 58,
 87: 63,
 88: 66,

In [26]:
my_cluster = Cluster((10, 20, 30), bitarray(10), 4)
print(type(my_cluster))
my_cluster.bits
my_cluster.stat_parts()

<class 'cluster.Cluster'>


{}